In [ ]:
import pandas as pd
import numpy as np

In [ ]:
# Attributes Information:

data_info = pd.read_csv("UNUSW-NB15_features.csv", encoding = "ISO-8859-1", header=None).values
data_info

In [ ]:
# Read the data available in 4 csv files using temporary data frames

dataframe1 = pd.read_csv("UNSW-NB15_1.csv", header=None)
dataframe2 = pd.read_csv("UNSW-NB15_2.csv", header=None)
dataframe3 = pd.read_csv("UNSW-NB15_3.csv", header=None)
dataframe4 = pd.read_csv("UNSW-NB15_4.csv", header=None)

# Merge all four data frames into one 

dataframe = pd.concat([dataframe1, dataframe2, dataframe3, dataframe4])

In [ ]:
# Delete temporary data frames

del dataframe1
del dataframe2
del dataframe3
del dataframe4